# Assignment

-------------

**1) Complete this notebook and make a pull request:** 

Answer questions (Q) in the space provided (A) in this notebook. When finished, copy your notebook to the `Assignment/` directory and name it `nb-6.5-<Github-username>.ipynb`. Then make a pull request to the upstream repo. The entered answers in this notebook will be simply Markdown text where I want you to interpret and describe a block of code to better understand what it is doing. Much of this code you will have seen already. 


**2) Write an importable Python package, save as a repo, and test it here.**

The package should be written as we did in our last lession (`.py` files in a directory with a setup.py file so it can be installed with `pip`). Follow instructions at the end of this notebook for how to write your package. Test it here by importing the package and executing the code at the end. It should work and give correct answers, if not, continue working on it. When you have it completed save your package as a new Github repo named `seqlib`.

### The `seqlib` package

Together we are going to write several functions here that will make up your new package called `seqlib`. It will be your job to copy these functions, organize them into a Class, save the code into a `.py` file (you can use SublimeText if you're comfortable with it for much of this, or any text editor including the one in jupyter), package the files so they can be imported as a library, and test the package so that it accomplishes the tasks which are defined at the end of this notebook. First things first, though, let's write the functions. 

In [12]:
import numpy as np
import pandas as pd

### Q.  Describe what the `mutate()` function below does:


A. A base is subtracted from the set `("ACTG")` and `np.random.choice` is run using the remaining bases.

In [13]:
def mutate(base):
    diff = set("ACTG") - set(base)
    return np.random.choice(list(diff))

In [111]:
# test it
mutate("A")

'C'

### Q. Describe how the `seqdata()` function below works:
Annotate the code by inserting lines with comments as you read through the function to make sense of it. What is being created at each step and how is it used?

A. See annotations in code

In [10]:
def simulate(ninds, nsites):
    #Generating sequence data
    oseq = np.random.choice(list("ACGT"), size=nsites)
    arr = np.array([oseq for i in range(ninds)])
    #Adding mutations at a 0.1 frequency
    muts = np.random.binomial(1, 0.1, (ninds, nsites))
    for col in range(nsites):
        newbase = mutate(arr[0, col])
        mask = muts[:, col].astype(bool)
        arr[:, col][mask] = newbase
    #Adding missing data in the form of "N"
    missing = np.random.binomial(1, 0.1, (ninds, nsites))
    arr[missing.astype(bool)] = "N"
    #Returning the generated sequence data that contains mutations and missing bases
    return arr

In [113]:
seqs = simulate(6, 15)
print(seqs)

[['N' 'C' 'C' 'G' 'A' 'N' 'T' 'N' 'G' 'C' 'A' 'T' 'N' 'G' 'T']
 ['C' 'C' 'N' 'A' 'A' 'G' 'T' 'G' 'C' 'T' 'N' 'T' 'T' 'G' 'T']
 ['N' 'C' 'C' 'N' 'A' 'G' 'T' 'N' 'C' 'C' 'A' 'T' 'T' 'G' 'T']
 ['C' 'A' 'C' 'G' 'A' 'G' 'T' 'G' 'G' 'C' 'A' 'T' 'G' 'G' 'C']
 ['C' 'C' 'N' 'G' 'A' 'G' 'T' 'T' 'G' 'C' 'A' 'T' 'T' 'G' 'T']
 ['C' 'N' 'C' 'G' 'A' 'G' 'T' 'G' 'G' 'C' 'A' 'T' 'T' 'G' 'C']]


### **Q: Describe how the `filter_missing` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it find columns with missing (N) values in them? How might you mprove it?

A. See annotations in code below

In [12]:
def filter_missing(arr, maxfreq):
    #Creating a criteria to return a modified array that substracts columns that
    #have a frequency of missing base pairs defined by the user
    freqmissing = np.sum(arr == "N", axis=0) / arr.shape[0]
    #Returning the modified array
    return arr[:, freqmissing <= maxfreq]
#The function could be improved by creating a copy of the array similar to what is done in filter_maf

In [115]:
filter_missing(seqs, 0.1)

array([['A', 'T', 'G', 'C', 'T', 'G', 'T'],
       ['A', 'T', 'C', 'T', 'T', 'G', 'T'],
       ['A', 'T', 'C', 'C', 'T', 'G', 'T'],
       ['A', 'T', 'G', 'C', 'T', 'G', 'C'],
       ['A', 'T', 'G', 'C', 'T', 'G', 'T'],
       ['A', 'T', 'G', 'C', 'T', 'G', 'C']],
      dtype='<U1')

### **Q: Describe how the `filter_maf` function works:**
Annotate the code by inserting lines with comments as you read through the function to make sense of it. How does it calculate minor allele frequencies? Why does it use copy?

A. filter_maf creates a modified array that contains only columns with a user-defined minfreq

In [13]:
def filter_maf(arr, minfreq):
    #Summing true values and divide by len of columns (shape[0]) to get frequency and store as freqs
    freqs = np.sum(arr != arr[0], axis=0) / arr.shape[0]
    #We create a copy to that we don't overwrite the original array
    maf = freqs.copy()
    #Subselect sites with major freq (>0.5) and modify to be 1-value
    maf[maf > 0.5] = 1 - maf[maf > 0.5]
    #Returning the new array that is filtered by only columns that have a greater minor allele frequency
    #than the user-defined minfreq
    return arr[:, maf > minfreq]

In [117]:
filter_maf(seqs, 0.1)

array([['N', 'C', 'C', 'G', 'N', 'N', 'G', 'C', 'A', 'N', 'T'],
       ['C', 'C', 'N', 'A', 'G', 'G', 'C', 'T', 'N', 'T', 'T'],
       ['N', 'C', 'C', 'N', 'G', 'N', 'C', 'C', 'A', 'T', 'T'],
       ['C', 'A', 'C', 'G', 'G', 'G', 'G', 'C', 'A', 'G', 'C'],
       ['C', 'C', 'N', 'G', 'G', 'T', 'G', 'C', 'A', 'T', 'T'],
       ['C', 'N', 'C', 'G', 'G', 'G', 'G', 'C', 'A', 'T', 'C']],
      dtype='<U1')

### Q: What order should these functions be applied, does it matter?

A. I cannot see a reason why this would matter other than that filter_maf creates a freq.copy to not modify the original array but filter_missing does not. Thus, if we run filter_missing first, we are modifying the original array and then running filter_maf. If we run filter_maf first we will run both functions on a copy of the array.

In [120]:
filter_missing(filter_maf(seqs, 0.1), 0.1)

array([['G', 'C', 'T'],
       ['C', 'T', 'T'],
       ['C', 'C', 'T'],
       ['G', 'C', 'C'],
       ['G', 'C', 'T'],
       ['G', 'C', 'C']],
      dtype='<U1')

In [121]:
filter_maf(filter_missing(seqs, 0.1), 0.1)


array([['G', 'C', 'T'],
       ['C', 'T', 'T'],
       ['C', 'C', 'T'],
       ['G', 'C', 'C'],
       ['G', 'C', 'T'],
       ['G', 'C', 'C']],
      dtype='<U1')

### Q: Describe how `calculate_statistics()` works

A. See below

In [16]:
def calculate_statistics(arr):
    #Calculating the mean nucleotide diversity of the arrays by calculating the variance
    #and then dividing it by the mean
    nd = np.var(arr == arr[0], axis=0).mean()
    #Calculating mean minor allele frequency by finding mutated sites
    #and dividing by the len of columns (shape[0])
    mf = np.mean(np.sum(arr != arr[0], axis=0) / arr.shape[0])
    #Calculating invariance
    #The only issue I have is that since the != operator is used, it seems that this line is
    #actually calculating variance, not invariance
    inv = np.any(arr != arr[0], axis=0).sum()
    #Calculating variance by subtracting the number of invariant sites
    var = arr.shape[1] - inv
    #Returning a pandas series"
    return pd.Series(
        {"mean nucleotide diversity": nd,
         "mean minor allele frequency": mf,
         "invariant sites": inv,
         "variable sites": var,
        })

In [38]:
calculate_statistics(seqs)

invariant sites                10.000000
mean minor allele frequency     0.244444
mean nucleotide diversity       0.103704
variable sites                  5.000000
dtype: float64

### Instructions: Write a `seqlib` Class object

I started writing the bare bones of it below. You should write it so that it can be executed as described below to perform all of the functions we defined above, and so that its attributes can be accessed. Save this class object in a `.py` file and make it into an importable package called `seqlib`. You can write and test your object in this notebook if you like, but it must be saved separately in a `.py` file and be imported. You cannot execute the code at the end using your object defined here in the notebook. When finished save your package to GitHub as a repo just like we did with the `helloworld` package. You do not need to write a CLI script like we did for the `helloworld` package, we will only be using the Python API here. See the examples below for **how you should write your Class object**. It should be able to run in the way written below, so look at that code and think about how you would write a Class object that can do that. 

While you can mostly copy the functions from above, you will need to modify them slightly to access information about the Class object using *self*. For example, the `simulate()` function below takes self as a first argument and can access `self.inds` and `self.nsites` from that, so we do not need to provide those as arguments to the `simulate` function like we did above. 

In [69]:
class Seqlib:

    def __init__(self, ninds, nsites):
        self.ninds = ninds
        self.nsites = nsites
        self.seqs = self.simulate()

    def simulate(self):
        oseq = np.random.choice(list("ACGT"), size=self.nsites)
        arr = np.array([oseq for i in range(self.ninds)])
        muts = np.random.binomial(1, 0.1, (self.ninds, self.nsites))
        for col in range(self.nsites):
            newbase = np.random.choice(list(set("ACTG") - set(arr[0, col])))
            mask = muts[:, col].astype(bool)
            arr[:, col][mask] = newbase
        missing = np.random.binomial(1, 0.1, (self.ninds, self.nsites))
        arr[missing.astype(bool)] = "N"
        return arr
    
    def maf(self):
        freqs = np.sum(self.seqs != self.seqs[0], axis=0) / self.seqs.shape[0]
        maf = freqs.copy()
        maf[maf > 0.5] = 1 - maf[maf > 0.5]
        print(maf)
    
    def filter(self, minmaf, maxmissing):
        freqs = np.sum(self.seqs != self.seqs[0], axis=0) / self.seqs.shape[0]
        maf = freqs.copy()
        maf[maf > 0.5] = 1 - maf[maf > 0.5]
        freqmissing = np.sum(self.seqs == "N", axis=0) / self.seqs.shape[0]
        self.seqs[:, maf > minmaf]
        return self.seqs[:, freqmissing <= maxmissing]
        
    def calculate_statistics(self):
        nd = np.var(self.seqs == self.seqs[0], axis=0).mean()
        mf = np.mean(np.sum(self.seqs != self.seqs[0], axis=0) / self.seqs.shape[0])
        inv = np.any(self.seqs != self.seqs[0], axis=0).sum()
        var = self.seqs.shape[1] - inv
        return pd.Series(
            {"mean nucleotide diversity": nd,
             "mean minor allele frequency": mf,
             "invariant sites": inv,
             "variable sites": var,
            })
    

## Test your package
The package should be globally importable (you ran `pip install .` or `pip install -e .` to install it), and it should be able to execute the following code without error. 

In [1]:
import seqlib

In [15]:
# init a Seqlib Class object
seqs = seqlib.Seqlib(ninds=6, nsites=10)
seqs.simulate()

array([['C', 'G', 'T', 'C', 'C', 'T', 'C', 'A', 'G', 'T'],
       ['C', 'G', 'T', 'A', 'T', 'T', 'C', 'A', 'G', 'N'],
       ['N', 'G', 'T', 'C', 'C', 'T', 'T', 'A', 'G', 'T'],
       ['N', 'A', 'T', 'C', 'C', 'T', 'C', 'A', 'G', 'T'],
       ['C', 'N', 'T', 'C', 'N', 'T', 'C', 'A', 'G', 'T'],
       ['C', 'G', 'T', 'C', 'C', 'T', 'T', 'A', 'G', 'T']],
      dtype='<U1')

In [8]:
# access attributes from the object
print(seqs.ninds, seqs.nsites)

6 10


In [4]:
# returns the MAF of the array as an array of floats
seqs.maf()

[ 0.33333333  0.33333333  0.16666667  0.16666667  0.16666667  0.          0.
  0.16666667  0.16666667  0.        ]


In [10]:
# return a view of the filtered sequence array by applying a new function 
# called `filter()` that applies both the maf and missing filter functions
seqs.filter(minmaf=0.1, maxmissing=0.0)

array([['A', 'C', 'A', 'C'],
       ['A', 'C', 'A', 'G'],
       ['C', 'C', 'A', 'G'],
       ['A', 'C', 'A', 'C'],
       ['A', 'C', 'A', 'C'],
       ['A', 'C', 'A', 'C']],
      dtype='<U1')

In [5]:
# calculate statistics for an array with the results returned as a DataFrame
seqs.calculate_statistics()

invariant sites                7.000000
mean minor allele frequency    0.216667
mean nucleotide diversity      0.113889
variable sites                 3.000000
dtype: float64

In [ ]:
# calculate statistics for an array after filtering it
seqs.filter(minmaf=0.1, maxmissing=0.0).calculate_statistics()